Справочный материал:

Основы_Python_7_классы.pdf

ML_4_2_реализация_обучения_персептрона.pdf

ML_4_3_ирисы_Фишера.pdf

ML_4_4_тренировка_персептрона.pdf

ML_4_5_ADALINE.pdf

ML_4_6_реализация_ADALINE.pdf

ML_4_7_тренировка_ADALINE.pdf

ML_4_8_ADALINE_стохастический_градиентный_спуск.pdf

1. Используя рассмотренный в ML_4_2_реализация_обучения_персептрона.pdf класс Perceptron в
качестве базового класса, разработать дочерний класс AdalineGD, реализующий модель адаптивного линейного
нейрона. Конструктор класса должен принимать аргумент, осуществляющий переключение между пакетным и
стохастическим градиентным спуском.
2. Загрузить набор данных цветков ириса из модуля datasets библиотеки scikit-learn. Сформировать
обучающую выборку, включающую результаты измерений длины чашелистика (sepal length) и длины лепестка (petal
length) для ириса щетинистого (lris setosa) и ириса разноцветного (Iris versicolor).
Отобразить на графике расположение данных в пространстве признаков.
3. Создать модель персептрона базового класса и обучить её на имеющихся данных при использовании скорости
обучения eta=0.01 и длительности обучения n_iter=10. Построить график изменения количества ошибочных
классификаций в процессе обучения. Построить график области решений при помощи функции
plot_decision_regions(), разработанной ранее на практических занятиях.
4. Создать модель ADALINE, использующую для обучения пакетный градиентный спуск, и обучить её при
значениях гиперпараметров длительности и скорости обучения n_iter=50, eta=0.0001. Построить график области
решений при помощи функции plot_decision_regions().
Обучить созданную модель при значениях гиперпараметров n_iter=10, eta=0.01, а также n_iter=10,
eta=0.0001. Построить графики изменения среднеквадратической ошибки в процессе обучения. Пояснить, почему в
одном случае модель успешно обучается, а в другом случае ошибка возрастает с каждой итерацией.
5. Создать модель ADALINE, использующую для обучения стохастический градиентный спуск, и обучить её при
значениях гиперпараметров n_iter=15, eta=0.01. Построить графики изменения среднеквадратической ошибки в
процессе обучения, а также график области решений при помощи функции plot_decision_regions().
6. Сделать выводы по проделанной работе.

In [1]:
# Импотр библиотек
import numpy as np

In [ ]:
# TODO: посмотреть что такое np.dot, np.where, функция обучения перцептрона

class Perceptron():
    def __init__(self, eta=0.1, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter

    def netInput(self, x):
        return np.dot(x,  self.w_[1:], self.w_[0])

    def predict(self, x):
        return np.where(self.net_input(x) >= 0.0, 1, -1)

    def fit(self, x, y):
        self.w_ = np.zeros(1 + x.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(x, y):
                update = self.eta * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self